# Weight-Setting in Weighted Jensen-Shannon Divergence

Goal: We know that using the weight vector of `[5,4,3,2,1]` makes the Jensen-Shannon divergence only *ordinal* tool to compare two distribution, while proportionality is not achieved. Our goal is to find out how this set of weights distorts the four important areas in quadrant plot, namely whether it consist of rectangular (or even square) areas or whether a more complicated distoriton occurs.  

Approach: Examine the effect of weights and project the divergence of simulated distributions to quadrant plot.

## Formula for weighted Jensen-Shannon divergence

My own implementation - see bottom for trials with ready packages.

In [38]:
import numpy as np
from scipy.spatial import distance

In [4]:
def djs(p, q, weights=[5,4,3,2,1]):
    """
    My own implementation of standardized weighted Jensen-Shannon divergence.
    
    Parameters
    ----------
    p,q : sequence
        Define the (discrete) distributions. ``pk[i]`` is the normalized probability of event ``i``.
    wgh : sequence, optional
        Weights to be applied, wgh[i] relates to p[i] and q[i]
    Returns
    -------
    djs : float
        The calculated divergence.
    """
    
    #set weights if no supplied
    if weights == None:
        weights = [4,3,2,1,0]
        
    # mixture
    mx = [p + q for p,q in zip(p,q)] # NB: no normalization here!

    # sum
    part1 = [wi * pi * np.log(2*pi / mxi) for wi,pi,mxi in zip(weights,p,mx) if mxi>0 and pi>0]
    part2 = [wi * qi * np.log(2*qi / mxi) for wi,qi,mxi in zip(weights,q,mx) if mxi>0 and qi>0]
    djs = 1/2*(np.array(part1).sum() + np.array(part2).sum())
    part1, part2, djs, djs == np.log(2)

    # result with normalziation: NB: the factor only applies for weights of [5,4,3,2,1]!
    return djs/(3*np.log(2))*np.sign(djs)

Sample distributions:

In [30]:
ideal =     [0,0,0,0,1]
medioplus = [0,0,0,1,0]
medio =     [0,0,1,0,0]
mediomin  = [0,1,0,0,0]
worst =     [1,0,0,0,0]

Effect of weights:

In [6]:
djs(ideal, worst)

1.0

In [7]:
djs(ideal, worst, weights=[777]*5)

259.00000000000006

Non-linearity of weights:

In [31]:
djs(ideal, worst)

1.0

In [32]:
djs(ideal, mediomin)

0.83333333333333337

In [33]:
djs(ideal, medio)

0.66666666666666674

In [34]:
djs(ideal, medioplus)

0.5

In [35]:
djs(ideal, ideal)

0.0

## Simulated distributions
Make up more realistic distributions consisting of more than one peak, compare the `djs` between them and the ideal one and examine the ordinality. 

*skipping for the moment*

# Euclidean distance
*hypothesis*: Can we measure the distance between distributions using the most common metric in the 5D space.

In [39]:
distance.euclidean(ideal, ideal)

0.0

In [41]:
distance.euclidean(ideal, medioplus)

1.4142135623730951

In [42]:
distance.euclidean(ideal, medio)

1.4142135623730951

In [43]:
distance.euclidean(ideal, mediomin)

1.4142135623730951

In [40]:
distance.euclidean(ideal, worst)

1.4142135623730951

In [45]:
np.sqrt(2)

1.4142135623730951

conclusion: Suffers from the same issue as with plain divergence: each pair of 'corners' in the cube has the same distance.

# sand box: Efforts with other packages

In [47]:
from scipy.special import (comb, chndtr, entr, rel_entr, xlogy, ive)

In [10]:
#https://gist.github.com/zhiyzuo/f80e2b1cfb493a5711330d271a228a3d
    
import numpy as np
import scipy as sp

def jsd(p, q, base=np.e):
    '''
        Implementation of pairwise `jsd` based on  
        https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence
    '''
    ## convert to np.array
    p, q = np.asarray(p), np.asarray(q)
    ## normalize p, q to probabilities
    p, q = p/p.sum(), q/q.sum()
    m = 1./2*(p + q)
    return sp.stats.entropy(p,m, base=base)/2. + sp.stats.entropy(q, m, base=base)/2.

In [15]:
p = [0,0,0,0,1] #ideal
q = [1,0,0,0,0] #worst
print(jsd(p,q))
print(jsd(p,q) == scipy.log(2))

0.69314718056
True


In [12]:
p = [0,0,0,0,1] #ideal
q = [0,0,1,0,0] #mediocre
jsd(p,q)

0.69314718055994529

In [20]:
# https://dit.readthedocs.io/en/latest/measures/divergences/jensen_shannon_divergence.html
import dit

In [21]:
import dit.divergences.jensen_shannon_divergence as jsdw

In [32]:
from dit import Distribution, ScalarDistribution
outcomes = ['A', 'B', 'C', 'D', 'E']
pmf = [1/5]*5
mydist = Distribution(outcomes, pmf)

In [38]:
from dit.divergences import jensen_shannon_divergence, kullback_leibler_divergence
# X = dit.ScalarDistribution(['red', 'blue'], [1/2, 1/2])
# Y = dit.ScalarDistribution(['blue', 'green'], [1/2, 1/2])
# jensen_shannon_divergence([X, Y])

In [42]:
p = Distribution(outcomes, [0,0,0,0,1]) #ideal
q = Distribution(outcomes, [1,0,0,0,0]) #worst
m = Distribution(outcomes, [0,0,1,0,0]) #mediocre
kullback_leibler_divergence(p, q)

inf

In [44]:
p = ScalarDistribution(outcomes, [0,0,0,0,1]) #ideal
q = ScalarDistribution(outcomes, [1,0,0,0,0]) #worst
m = ScalarDistribution(outcomes, [0,0,1,0,0]) #mediocre
wg = [5,4,3,2,1]
#jsdw(dists=[p, q], weights= wg)
#print(jsdw(p,q) == scipy.log(2))

In [46]:
jensen_shannon_divergence([p, q], weights=wg)

ditException: Length of `dists` and `weights` must be equal.

In [36]:
jensen_shannon_divergence([p, m])

1.0